<a href="https://colab.research.google.com/github/alxkzncoff/skillfactory_rds6_car_price_prediction_ml_dl/blob/main/model/cat_boost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL

In [3]:
!pip install catboost

     |████████████████████████████████| 66.2MB 57kB/s 


# DEFINE

In [5]:
# DEFINE
DATA_PATH                   = '/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl'
RANDOM_SEED                 = 42

# IMPORT

In [89]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostRegressor
from catboost import Pool
from catboost import cv
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [114]:
train = pd.read_csv(DATA_PATH+'/clear_train.csv')
test = pd.read_csv(DATA_PATH+'/clear_test.csv')

In [115]:
train.model_date = train.model_date.astype(str)
test.model_date = test.model_date.astype(str)

train.production_date = train.production_date.astype(str)
test.production_date = test.production_date.astype(str)

train.num_of_doors = train.num_of_doors.astype(str)
test.num_of_doors = test.num_of_doors.astype(str)

In [116]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6682 entries, 0 to 6681
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   body                 6682 non-null   object 
 1   brand                6682 non-null   object 
 2   color                6682 non-null   object 
 3   description          6682 non-null   object 
 4   engine_displacement  6682 non-null   float64
 5   engine_power         6682 non-null   float64
 6   fuel                 6682 non-null   object 
 7   mileage              6682 non-null   int64  
 8   model_date           6682 non-null   object 
 9   model_info           6682 non-null   object 
 10  name                 6682 non-null   object 
 11  num_of_doors         6682 non-null   object 
 12  price                6682 non-null   float64
 13  production_date      6682 non-null   object 
 14  transmission         6682 non-null   object 
 15  owners               6682 non-null   o

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   body                 1671 non-null   object 
 1   brand                1671 non-null   object 
 2   color                1671 non-null   object 
 3   description          1671 non-null   object 
 4   engine_displacement  1671 non-null   float64
 5   engine_power         1671 non-null   float64
 6   fuel                 1671 non-null   object 
 7   mileage              1671 non-null   int64  
 8   model_date           1671 non-null   object 
 9   model_info           1671 non-null   object 
 10  name                 1671 non-null   object 
 11  num_of_doors         1671 non-null   object 
 12  production_date      1671 non-null   object 
 13  transmission         1671 non-null   object 
 14  owners               1671 non-null   object 
 15  vehicle_title        1671 non-null   o

None

# DATA PREPARING

In [117]:
MMS = MinMaxScaler()
MMS_price = MinMaxScaler()
STDS = StandardScaler()

In [118]:
MMS.fit(train[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']])

MMS_price.fit(train[['price']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [119]:
train[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']] = MMS.transform(train[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']])

train[['price']] = MMS_price.transform(train[['price']])

display(train[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']])

display(train[['price']])

,engine_displacement,engine_power,mileage,tax,days_in_use
0,0.400000,0.349291,0.244999,0.420221,0.342100
1,0.366667,0.228723,0.182999,0.130174,0.236833
2,0.483333,0.409574,0.122732,0.473934,0.184163
3,0.233333,0.186170,0.150999,0.085308,0.236833
4,0.200000,0.150709,0.139999,0.066351,0.236833
...,...,...,...,...,...
6677,0.233333,0.132979,0.232776,0.045814,0.368398
6678,0.233333,0.203901,0.033699,0.090574,0.105267
6679,0.233333,0.120567,0.199999,0.043233,0.315801
6680,0.400000,0.416667,0.071999,0.480253,0.105267


,price
0,0.012615
1,0.023251
2,0.043379
3,0.021768
4,0.021556
...,...
6677,0.010539
6678,0.067745
6679,0.009691
6680,0.070711


# SPLIT

In [120]:
X = train.drop(columns=['price', 'description'])
y = train['price']

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

In [109]:
gsctb = CatBoostRegressor()

params_grid = {'iterations':[1500, 2000, 4000, 5500],
               'learning_rate':[0.5, 0.1, 0.05, 0.03, 0.005],
               'l2_leaf_reg':[1, 2, 4, 6, 8],
               'bootstrap_type': ['Bayesian','Bernoulli']}

grid_search_result = gsctb.grid_search(params_grid,
                                     train_data,
                                     cv=5,
                                     partition_random_seed=RANDOM_SEED,
                                     calc_cv_statistics=True,
                                     search_by_train_test_split=True,
                                     refit=True,
                                     shuffle=True,
                                     train_size=0.8,
                                     verbose=500,
                                     plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 0.02168010015
bestIteration = 180

0:	loss: 0.0216801	best: 0.0216801 (0)	total: 28s	remaining: 1h 33m 1s

bestTest = 0.02167111132
bestIteration = 204


bestTest = 0.01758113602
bestIteration = 1336


bestTest = 0.01727303693
bestIteration = 1385


bestTest = 0.01686095674
bestIteration = 1491


bestTest = 0.01705941128
bestIteration = 1498


bestTest = 0.01749688585
bestIteration = 1496


bestTest = 0.01760635785
bestIteration = 1499


bestTest = 0.01980787758
bestIteration = 1498


bestTest = 0.0197833543
bestIteration = 1498


bestTest = 0.02090404849
bestIteration = 223


bestTest = 0.02281607474
bestIteration = 194


bestTest = 0.01710341518
bestIteration = 1493


bestTest = 0.01775592799
bestIteration = 1400


bestTest = 0.01751988412
bestIteration = 1496


bestTest = 0.01759210663
bestIteration = 1496


bestTest = 0.01759073721
bestIteration = 1496


bestTest = 0.01770734685
bestIteration = 1498


bestTest = 0.02034107638
bestIteration = 1499


bestTest = 0.02023259

In [110]:
grid_search_result['params']

{'bootstrap_type': 'Bayesian',
 'iterations': 4000,
 'l2_leaf_reg': 1,
 'learning_rate': 0.05}

In [122]:
train_data = Pool(data = X_train,
                  label = y_train,
                  cat_features =  ['body', 'brand', 'color', 'fuel', 'model_date', 'model_info', 'name', 'num_of_doors',
                                   'production_date', 'transmission', 'owners', 'vehicle_title', 'drive_type'])

test_data = Pool(data = X_test,
                 label = y_test,
                  cat_features =  ['body', 'brand', 'color', 'fuel', 'model_date', 'model_info', 'name', 'num_of_doors',
                                   'production_date', 'transmission', 'owners', 'vehicle_title', 'drive_type'])

# MODEL

In [123]:
ctb = CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.05,
                         iterations=4000,
                         l2_leaf_reg=1,
                         depth=6,
                         bootstrap_type = 'Bayesian', # Bayesian Bernoulli
#                          subsample = 0.8,
                         one_hot_max_size = 5,
                         random_seed=42)
                        #  od_type='Iter',
                        #  od_wait=500)

model = ctb.fit(train_data,
                eval_set=test_data,
                verbose_eval=1000,
                use_best_model=True,
                plot=True)
print(model)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0444445	test: 0.0447610	best: 0.0447610 (0)	total: 29.7ms	remaining: 1m 58s
1000:	learn: 0.0077031	test: 0.0100373	best: 0.0100361 (993)	total: 11.9s	remaining: 35.7s
2000:	learn: 0.0063318	test: 0.0098984	best: 0.0098958 (1755)	total: 25s	remaining: 25s
3000:	learn: 0.0056420	test: 0.0098120	best: 0.0098067 (2964)	total: 39.1s	remaining: 13s
3999:	learn: 0.0051537	test: 0.0098910	best: 0.0098067 (3021)	total: 54.6s	remaining: 0us

bestTest = 0.009806671398
bestIteration = 3021

Shrink model to first 3022 iterations.


# PREDICTION

In [124]:
test.drop(columns=['description'], inplace=True)

In [125]:
MMS.fit(test[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [126]:
test[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']] = MMS.transform(test[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']])

test[['engine_displacement', 'engine_power', 'mileage', 'tax', 'days_in_use']]

,engine_displacement,engine_power,mileage,tax,days_in_use
0,0.400000,0.198664,0.349999,0.114906,0.763167
1,0.400000,0.353923,0.014999,0.433520,0.184163
2,0.316667,0.195326,0.166599,0.113637,0.315801
3,0.233333,0.058431,0.146999,0.021810,0.342100
4,0.233333,0.148581,0.023999,0.068065,0.026299
...,...,...,...,...,...
1666,0.233333,0.138564,0.075999,0.065138,0.210534
1667,0.400000,0.195326,0.120380,0.113637,0.078896
1668,0.133333,0.081803,0.025789,0.026592,0.105267
1669,0.233333,0.138564,0.103999,0.065138,0.184163


In [127]:
predict_data = Pool(data = test,
                    cat_features =  ['body', 'brand', 'color', 'fuel', 'model_date', 'model_info', 'name', 'num_of_doors',
                                     'production_date', 'transmission', 'owners', 'vehicle_title', 'drive_type'])

In [128]:
predict_submission = MMS_price.inverse_transform(model.predict(predict_data).reshape(-1,1))
predict_submission

array([[ 596935.33180946],
       [2576027.35494361],
       [ 626210.62980218],
       ...,
       [1490723.76310959],
       [1050372.92725822],
       [ 975687.93772615]])

# MAKE SUBMISSION


In [129]:
sample_submission = pd.read_csv('/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl/sample_submission.csv')

In [130]:
# Результат как есть
sample_submission['price'] = predict_submission
sample_submission.to_csv('/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl/sub.csv', index=False)
display(sample_submission.head(10)) # Пока лучший результат

# Результат с округлением
sample_submission['price'] = np.round(predict_submission, -4)
sample_submission.to_csv('/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl/sub_round.csv', index=False)
display(sample_submission.head(10))

# Результат с округлением и умножением на коэффициент 0.94
sample_submission['price'] = np.round(predict_submission, -4)*0.94
sample_submission.to_csv('/content/drive/My Drive/skill_factory_car_price_prediction_ml_dl/sub_round_coef.csv', index=False)
display(sample_submission.head(10))

,sell_id,price
0,1099427284,5.969353e+05
1,1096405886,2.576027e+06
2,1100195294,6.262106e+05
3,1099827282,4.903081e+05
4,1100076198,2.432865e+06
5,1090159352,3.004081e+06
6,1098987386,1.416537e+06
7,1098639040,1.324897e+06
8,1099933039,1.037099e+06
9,1099565572,4.853569e+05


,sell_id,price
0,1099427284,600000.0
1,1096405886,2580000.0
2,1100195294,630000.0
3,1099827282,490000.0
4,1100076198,2430000.0
5,1090159352,3000000.0
6,1098987386,1420000.0
7,1098639040,1320000.0
8,1099933039,1040000.0
9,1099565572,490000.0


,sell_id,price
0,1099427284,564000.0
1,1096405886,2425200.0
2,1100195294,592200.0
3,1099827282,460600.0
4,1100076198,2284200.0
5,1090159352,2820000.0
6,1098987386,1334800.0
7,1098639040,1240800.0
8,1099933039,977600.0
9,1099565572,460600.0
